## IZV – projekt 2025 (varianta Nehody): Úkol 2 – Test hypotézy

Tento notebook ověřuje na hladině významnosti \(lpha = 0.05\) (95% jistota) dvě hypotézy nad daty *Statistika nehodovosti Policie ČR*.

**Předpoklad:** soubory `accidents.pkl.gz` a `vehicles.pkl.gz` jsou ve stejném adresáři jako tento notebook.

Pozn.: Data používají kódování neznámých hodnot (např. u `int` bývá `-1`, u řetězců prázdný řetězec).

In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency, mannwhitneyu

DATA_DIR = Path('.')
ACC_PATH = DATA_DIR / 'accidents.pkl.gz'
VEH_PATH = DATA_DIR / 'vehicles.pkl.gz'

if not ACC_PATH.exists() or not VEH_PATH.exists():
    raise FileNotFoundError(
        "V adresáři notebooku musí být accidents.pkl.gz a vehicles.pkl.gz"
    )

acc = pd.read_pickle(ACC_PATH)
veh = pd.read_pickle(VEH_PATH)

# datum nehody: v zadání bývá jako řetězec v p2a (DD.MM.YYYY); někdy je už i 'date'
if 'date' in acc.columns and np.issubdtype(acc['date'].dtype, np.datetime64):
    acc_date = acc['date']
else:
    acc_date = pd.to_datetime(acc['p2a'], format='%d.%m.%Y', errors='coerce')

acc = acc.copy()
acc['__year'] = acc_date.dt.year

acc.shape, veh.shape

## Hypotéza 1

**H0:** Na silnicích 1. třídy se při nehodách umíralo se stejnou pravděpodobností jako na silnicích 3. třídy.

Použijeme \(\chi^2\) test nezávislosti nad kontingenční tabulkou:

- **typ komunikace** (`p36`: silnice I. vs silnice III.) ×
- **alespoň 1 usmrcený** (`p13a > 0`).

V poskytnutých datech odpovídají kódy:
- `p36 == 2` … silnice I. třídy
- `p36 == 6` … silnice III. třídy

In [ ]:
ROAD_I = 2
ROAD_III = 6

h1 = acc.loc[acc['p36'].isin([ROAD_I, ROAD_III])].copy()

# "umíralo" → alespoň 1 usmrcený při nehodě
h1['death'] = (h1['p13a'] > 0).astype(int)

h1['road'] = np.where(h1['p36'] == ROAD_I, 'I. třída', 'III. třída')
ct1 = pd.crosstab(h1['road'], h1['death'])
ct1.columns = ['bez úmrtí', 'alespoň 1 úmrtí']
ct1

In [ ]:
chi2_1, p_1, dof_1, exp_1 = chi2_contingency(ct1, correction=False)
exp1 = pd.DataFrame(exp_1, index=ct1.index, columns=ct1.columns)

death_rate = (ct1['alespoň 1 úmrtí'] / ct1.sum(axis=1)).rename('pravděpodobnost úmrtí')

exp1, death_rate, {'chi2': float(chi2_1), 'dof': int(dof_1), 'p_value': float(p_1)}

### Závěr (Hypotéza 1)

- Pokud `p_value < 0.05`, zamítáme H0 a tvrdíme, že pravděpodobnost úmrtí se liší.
- Pokud `p_value >= 0.05`, H0 nezamítáme (v datech nevidíme statisticky významný rozdíl).

Směr rozdílu (kde je úmrtnost vyšší) je vidět z `pravděpodobnost úmrtí`.

### Doplňující část k Hypotéze 1: následky na zdraví (I. třída vs dálnice)

Zadání dále požaduje určit, zda nehody na silnicích 1. třídy vedly **častěji / méně často** k nehodě s následky na zdraví (`p9`) než na dálnicích (`p36`).

Použijeme opět \(\chi^2\) test nezávislosti pro:

- `p36 == 2` (silnice I. třídy) vs `p36 == 1` (dálnice)
- `injury = (p9 == 1)`

Směr (častěji/méně často) určíme porovnáním pozorovaných četností s `expected` (očekávanými) z \(\chi^2\) testu a také porovnáním podílů.

In [ ]:
HIGHWAY = 1

h1b = acc.loc[acc['p36'].isin([ROAD_I, HIGHWAY])].copy()

# interpretace p9: v poskytnutých datech nabývá typicky {1,2}
# p9==1 → nehoda s následky na zdraví; p9==2 → bez následků
h1b['injury'] = (h1b['p9'] == 1).astype(int)

h1b['road'] = np.where(h1b['p36'] == ROAD_I, 'I. třída', 'Dálnice')
ct1b = pd.crosstab(h1b['road'], h1b['injury'])
ct1b.columns = ['bez následků', 's následky (p9==1)']
ct1b

In [ ]:
chi2_1b, p_1b, dof_1b, exp_1b = chi2_contingency(ct1b, correction=False)
exp1b = pd.DataFrame(exp_1b, index=ct1b.index, columns=ct1b.columns)

inj_rate = (ct1b['s následky (p9==1)'] / ct1b.sum(axis=1)).rename('pravděpodobnost následků')

# jednoduché porovnání: (pozorováno - očekáváno) pro sloupec 's následky'
delta_injury = (ct1b['s následky (p9==1)'] - exp1b['s následky (p9==1)']).rename('obs - expected (s následky)')

exp1b, inj_rate, delta_injury, {'chi2': float(chi2_1b), 'dof': int(dof_1b), 'p_value': float(p_1b)}

**Interpretace směru:**

- Pokud je `obs - expected (s následky)` pro *I. třídu* kladné a pro *Dálnici* záporné (a zároveň `p_value < 0.05`), pak nehody na silnicích I. třídy vedou k následkům **častěji** než na dálnicích.
- Opačný vzor znamená, že vedou k následkům **méně často**.

Kromě toho lze směr ověřit i porovnáním `pravděpodobnost následků`.

## Hypotéza 2

**H0:** Škoda při nehodách značky X je stejná jako při nehodách značky Y.

**H1 (jednostranná):** Škoda při nehodách značky X je **nižší** než při nehodách značky Y a rozdíl je statisticky významný.

Pracujeme s:
- `p45a` … značka vozidla (kód)
- `p53` … škoda na vozidle

Rozdělení škod bývá silně nesymetrické (dlouhý pravý ocas), proto použijeme **neparametrický Mann–Whitney U test**.

Požadavek zadání: vybrat **dvě různé dvojice značek**:
- jednu, kde lze rozhodnout (X < Y statisticky významně),
- jednu, kde **nelze rozhodnout** (p-hodnota \(\ge 0.05\)).

In [ ]:
veh2 = veh.copy()

# odfiltruj neznámé/nesmyslné hodnoty
veh2 = veh2.loc[veh2['p45a'].notna() & (veh2['p45a'] != 0)]
veh2 = veh2.loc[veh2['p53'] >= 0]

veh2[['p45a', 'p53']].describe(include='all')

In [ ]:
# náhled na distribuci škod (kvantily)
qs = veh2['p53'].quantile([0.5, 0.75, 0.9, 0.95, 0.99])
qs

In [ ]:
# vyber nejčastější značky pro stabilní testy
TOP_N = 25
MIN_N_PER_BRAND = 300

counts = veh2['p45a'].value_counts()
top_brands = counts[counts >= MIN_N_PER_BRAND].head(TOP_N)

top_brands

In [ ]:
brand_stats = (
    veh2.loc[veh2['p45a'].isin(top_brands.index)]
    .groupby('p45a')['p53']
    .agg(n='size', median='median', mean='mean')
    .sort_values('median')
)
brand_stats

In [ ]:
def mwu_less(x: pd.Series, y: pd.Series):
    # H1: X < Y
    u, p = mannwhitneyu(x, y, alternative='less')
    return float(u), float(p)

# --- 1) найдём пару, где X<Y значимо ---
brands = list(map(float, brand_stats.index.tolist()))

pair_significant = None
# пробуем самый низкий медиан vs самый высокий медиан
for low in brands[:10]:
    for high in brands[-10:]:
        if low == high:
            continue
        x = veh2.loc[veh2['p45a'] == low, 'p53']
        y = veh2.loc[veh2['p45a'] == high, 'p53']
        if x.median() >= y.median():
            continue
        u, p = mwu_less(x, y)
        if p < 0.05:
            pair_significant = {
                'X': low,
                'Y': high,
                'nX': int(len(x)),
                'nY': int(len(y)),
                'medianX': float(x.median()),
                'medianY': float(y.median()),
                'U': u,
                'p_value': p,
            }
            break
    if pair_significant is not None:
        break

pair_significant

In [ ]:
# --- 2) найдём пару, где на alpha=0.05 нельзя решить (p>=0.05) ---
# берём пары с очень близкими медианами
pairs = []
for i in range(len(brands)):
    for j in range(i + 1, len(brands)):
        b1, b2 = brands[i], brands[j]
        x = veh2.loc[veh2['p45a'] == b1, 'p53']
        y = veh2.loc[veh2['p45a'] == b2, 'p53']
        med_diff = float(abs(x.median() - y.median()))
        # тестируем "b1 < b2" (если медианы равны/почти равны, часто будет p>=0.05)
        u, p = mwu_less(x, y)
        pairs.append((med_diff, p, b1, b2, int(len(x)), int(len(y)), float(x.median()), float(y.median()), u))

pairs.sort(key=lambda t: t[0])

pair_inconclusive = None
for med_diff, p, b1, b2, n1, n2, m1, m2, u in pairs[:80]:
    if p >= 0.05:
        pair_inconclusive = {
            'X': b1,
            'Y': b2,
            'nX': n1,
            'nY': n2,
            'medianX': m1,
            'medianY': m2,
            'U': float(u),
            'p_value': float(p),
            'median_diff': float(med_diff),
        }
        break

pair_inconclusive

### Závěr (Hypotéza 2)

- Pro dvojici `pair_significant` platí: pokud `p_value < 0.05`, zamítáme H0 ve prospěch H1 (škoda X je **nižší** než škoda Y).
- Pro dvojici `pair_inconclusive` platí: pokud `p_value >= 0.05`, na hladině \(lpha=0.05\) **nelze rozhodnout**, zda je škoda X nižší než Y.

Pozn.: V notebooku používáme kódy značek (`p45a`).

In [ ]:
# Shrnutí do jedné tabulky
summary_out = pd.DataFrame([
    {'case': 'significant (X<Y)', **(pair_significant or {})},
    {'case': 'inconclusive', **(pair_inconclusive or {})},
])
summary_out